In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('2021-01-07.csv',parse_dates=['Date'],index_col='Date')
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)

In [3]:
trading_days=250                  #Trading days per year (automated)

In [4]:
returnsh=df.pct_change()                  #Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)           #calculating daily returns of the stocks in the portfolio

In [5]:
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.004116,-0.017328,-0.003911,-0.003210,0.009825,0.024238,-0.013265,-0.004311,0.034958,-0.013568,...,-0.002317,0.043351,-0.026372,-0.013417,-0.007547,0.004776,-0.006674,0.001050,0.020061,0.000246
2,-0.004670,0.010079,0.022950,0.003701,-0.019795,0.009395,-0.015999,-0.003027,0.053465,-0.020043,...,-0.020953,0.004588,-0.001619,-0.006128,-0.006777,0.057034,-0.002407,-0.019236,-0.009251,0.000861


In [6]:
returnso=returnsh.copy()                  #this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
    for i in range(0,len(df)):
      if returnso[cols][i] > 0:
        returnso[cols][i]=0               #Here, returnso would mean return considered for sortino ratio

In [7]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
0,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,0.00000,-0.017328,-0.003911,-0.00321,0.000000,0.0,-0.013265,-0.004311,0.0,-0.013568,...,-0.002317,0.0,-0.026372,-0.013417,-0.007547,0.0,-0.006674,0.000000,0.000000,0.0
2,-0.00467,0.000000,0.000000,0.00000,-0.019795,0.0,-0.015999,-0.003027,0.0,-0.020043,...,-0.020953,0.0,-0.001619,-0.006128,-0.006777,0.0,-0.002407,-0.019236,-0.009251,0.0


In [8]:
covmatsh=returnsh.cov()*trading_days      #Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso=returnso.cov()*trading_days      #Annualised covariance matrix calculated wrt returnso i.e. used to calculate sortino ratio

In [9]:
num_portfolios = 50000                   #initializing number of portfolios to 50000; referred from Wang et al (2020) (science direct)
num_assets = len(df.columns)              #initializing number of stocks/assets considered in the portfolio
risk_free_rate = 0.0358                   #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [10]:
#2021_chen etal_Mean–variance portfolio optimization using machine learning-based stock price prediction
#Repeat the process 50,000times. From a statistical point of view, 50,000 random portfolios cover most possible portfolios with different weights and aresufficiently representative

In [11]:
portfolio_returns = []                    #initializing an empty list for portfolio returns
portfolio_volatility =[]                  #initializing an empty list for portfolio risk
stock_weights =[]                         #initializing an empty list for portfolio weights
semi_deviation =[]                        #initializing an empty list for portfolio semi-deviation
sharpe =[]                                #initializing an empty list for portfolio sharpe ratio
sortino =[]                               #initializing an empty list for portfolio sortino ratio

In [12]:
def ratio(a,b,c):                         #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  return (a-c)/b                          #a => annual return, c => risk_free_rate, b => deviation (standard for sharpe, semi for sortino)

In [13]:
for single_portfolio in range(num_portfolios):                  #iterating forloop for 50000 times to generate 50000 portfolios
  weights = np.random.random(num_assets)                        #initializing random weights
  weights /= np.sum(weights)                                    #No Short Selling Allowed => weights add up to 1   "x = x+y" => "x+=y"    weights = weights/np.sum(weights)  
  returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
  varsh=np.dot(weights.T,np.dot(covmatsh,weights))              #calculating portfolio varience wrt calculating sharpe ratio
  varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sortino ratio
  volatility_temp = np.sqrt(varsh)                              #portfolio risk
  semi_temp = np.sqrt(varso)                                    #portfolio semi-deviation
  shtemp = ratio(returns_temp,volatility_temp,risk_free_rate)   #calculating sharpe ratio
  sotemp = ratio(returns_temp,semi_temp,risk_free_rate)         #calculating sortino ratio
  portfolio_returns.append(returns_temp)                       
  portfolio_volatility.append(volatility_temp)
  stock_weights.append(weights)
  sharpe.append(shtemp)
  sortino.append(sotemp)
  semi_deviation.append(semi_temp)

In [14]:
portfolio = {'Returns' : portfolio_returns, 'Standard Deviation' : portfolio_volatility, 'Semi-Deviation' : semi_deviation, 'Sharpe Ratio' : sharpe, 
             'Sortino Ratio' : sortino}    
#here, 'portfolio' is a dictionary which will be used to create dataframe where each row will be a portfolio

In [15]:
for counter,symbol in enumerate(df.columns):
  portfolio[symbol + " Weight"] = [Weight[counter] for Weight in stock_weights] 
#to the dictionary (named 'portfolio'), weights for each symbol are added in so as to be displayed in the dataframe

In [16]:
pc = pd.DataFrame(portfolio)         #making the final dataframe where data of 50000 portfolios is appended (subject to be saved, whose code is to be written)

In [17]:
pc=pc*100                                       #Converting everything to percentage
pc['Sharpe Ratio']=pc['Sharpe Ratio']/100       #leaving ratios as it is
pc['Sortino Ratio']=pc['Sortino Ratio']/100

In [18]:
#pc.to_csv('portfolios_by_MV.csv')  #saving the portfolios data

In [19]:
max_sharpe=pc['Sharpe Ratio'].max()                                             #Best optimised portfolio wrt sharpe ratio
max_sharpe_portfolio=pc.loc[pc['Sharpe Ratio'] == max_sharpe]
max_sharpe_portfolio

,Returns,Standard Deviation,Semi-Deviation,Sharpe Ratio,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
41349,139.538106,7.644963,4.439984,17.784011,30.621303,4.897021,3.864047,6.768306,2.097125,6.280279,...,2.70757,7.862983,2.458484,3.088744,3.70853,10.354179,0.045029,2.829117,6.785199,1.185043


In [20]:
max_sortino=pc['Sortino Ratio'].max()                                            #Best optimised portfolio wrt sortino ratio
max_sortino_portfolio=pc.loc[pc['Sortino Ratio'] == max_sortino]
max_sortino_portfolio

,Returns,Standard Deviation,Semi-Deviation,Sharpe Ratio,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
41409,140.692294,7.811534,2.870183,17.552544,47.771266,7.814202,2.601109,6.719151,7.765505,0.570514,...,0.638815,8.056061,0.727291,0.787016,3.549922,7.486257,0.349224,0.399644,0.134699,7.945119


In [21]:
#code for visualization is to be written

In [22]:
pc_sharpe=pc.drop(columns=['Sortino Ratio','Semi-Deviation'])

In [23]:
pc_sharpe_top10=pc_sharpe.sort_values(by=['Sharpe Ratio'],ascending=False).head(10)

In [24]:
pc_sharpe_top10.to_csv('Sharpe_Top10_MV.csv')

In [25]:
pc_sharpe_bottom10=pc_sharpe.sort_values(by=['Sharpe Ratio'],ascending=False).tail(10)

In [26]:
pc_sharpe_bottom10.to_csv('Sharpe_Bottom10_MV.csv')

In [27]:
pc_sharpe_bottom10


,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
20688,-1.412546,0.077379,-64.520744,6.505425,6.734501,0.244547,5.997558,3.098880,0.883388,6.304268,...,6.779694,3.764470,4.292962,4.877389,1.547305,5.581312,3.487418,5.972662,1.937687,5.194983
32436,0.556050,0.046557,-64.951232,6.133688,3.958216,6.227632,2.471146,6.848168,1.383402,2.895661,...,3.699729,1.466507,3.934546,5.051018,7.628071,4.349875,5.601616,5.551349,2.766611,6.249155
19950,-1.255753,0.069983,-69.098611,4.030696,8.702390,3.804546,0.646826,6.792832,3.147368,8.681347,...,6.509293,3.442279,5.412876,2.403573,5.992465,1.124549,4.806486,2.557601,2.234728,0.649181
25861,-1.163985,0.066575,-71.257978,5.928812,1.831159,1.035220,0.538832,3.714850,1.293558,1.889891,...,8.248439,1.981571,8.700067,7.620768,8.020382,8.153276,4.907770,6.905867,6.111429,0.443050
49327,0.662718,0.040444,-72.130937,2.057283,4.255543,6.366961,6.291056,3.141035,0.188524,5.355900,...,2.086863,6.148385,5.586323,6.425521,5.191212,1.537606,2.524104,4.552840,4.155783,5.988355
12873,0.368011,0.036743,-87.417551,6.232165,6.680555,5.046503,1.343859,0.043010,1.808052,3.345441,...,6.556340,5.980680,6.895822,5.956072,6.038518,3.860093,6.928077,4.248766,6.914088,5.380804
25882,0.508130,0.031055,-98.916075,6.048952,6.816096,4.853425,3.776548,3.569998,3.934621,5.914613,...,5.253184,2.927081,7.542752,7.112452,0.829242,3.520799,3.945940,0.850597,9.095483,1.648513
24553,-0.528639,0.032786,-125.317321,0.924865,2.500145,3.371679,5.574714,2.120366,0.181378,7.422288,...,7.678290,2.514344,2.222510,6.357596,6.827112,6.636814,3.717902,3.225659,4.222838,5.834236
4335,-0.227187,0.026298,-144.771329,3.171889,6.629566,2.879303,6.798863,6.368109,5.938221,1.930768,...,4.550282,0.249515,6.366420,1.453803,4.983864,2.694125,7.389501,5.299903,1.815576,3.905874
48436,-0.294244,0.016477,-235.125081,5.892683,5.597180,1.293415,7.027414,3.472292,1.734407,4.808864,...,1.412964,1.820843,7.979741,3.727802,0.462337,7.190195,6.101880,7.807824,6.853054,3.434914


In [28]:
pc_sharpe_top10

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
41349,139.538106,7.644963,17.784011,4.897021,3.864047,6.768306,2.097125,6.280279,6.074598,5.686135,...,2.707570,7.862983,2.458484,3.088744,3.708530,10.354179,0.045029,2.829117,6.785199,1.185043
2898,136.061345,7.457356,17.765190,8.641108,0.420333,4.019908,1.076548,0.556706,6.093699,0.650930,...,5.277464,4.428675,2.276317,5.691646,6.210873,8.354961,0.920717,0.928999,5.289471,1.831541
2748,131.811426,7.219615,17.761533,0.509683,2.185858,5.712642,0.403285,1.642495,6.000012,3.559384,...,3.473076,7.944064,1.765362,2.286652,0.905538,8.071294,7.197414,5.527476,3.834788,2.335825
27610,135.022632,7.401722,17.758386,2.680960,1.302068,8.368484,7.611643,2.479078,6.549986,0.353298,...,5.060577,7.260056,1.421244,3.344979,0.798986,7.377264,2.336252,0.154388,4.180205,3.030722
41969,130.716327,7.159799,17.756968,3.753523,1.392830,5.802097,10.310062,3.679573,9.932470,0.008455,...,5.051075,1.476451,1.582967,1.825077,0.174606,6.719704,2.618528,4.443869,8.950154,8.400271
18496,134.819306,7.392903,17.752065,5.548454,0.984592,4.099427,1.991696,1.375940,4.971837,1.332731,...,8.913736,6.492263,3.348502,2.432150,2.976909,9.474705,0.675100,1.226801,7.004291,0.980478
11911,132.482578,7.261365,17.751837,0.293392,1.611126,7.091383,4.684699,3.882979,7.152955,5.218740,...,6.089206,8.049418,3.281281,0.521812,2.844933,6.586216,7.703204,0.334707,0.325485,6.576550
6800,132.571227,7.267029,17.750202,7.179727,0.928808,4.369045,5.294544,1.111205,6.902260,2.898588,...,4.267035,8.441651,0.404169,3.504017,3.513166,8.293879,3.821856,1.521377,4.809722,8.003612
31938,129.371822,7.089941,17.742294,6.417149,4.021359,3.316657,4.841029,0.002733,6.490259,2.351569,...,1.253163,7.834498,2.251364,3.887269,2.623436,7.003924,1.165902,3.198681,0.333083,7.273746
504,128.648822,7.052743,17.733359,4.546980,4.495287,6.092880,1.481713,3.649478,4.798903,4.947058,...,2.000159,5.808433,2.554731,0.813588,2.791924,7.330770,5.783674,0.286903,8.538159,4.909583


In [29]:
sharpe_optimal_portfolio=pc_sharpe_top10.head(1)

In [30]:
sharpe_optimal_portfolio.to_csv('Sharpe_Optimal_MV.csv')

In [31]:
sharpe_optimal_portfolio.T

,41349
Returns,139.538106
Standard Deviation,7.644963
Sharpe Ratio,17.784011
ASIANPAINT Weight,4.897021
BAJFINANCE Weight,3.864047
BAJAJFINSV Weight,6.768306
BRITANNIA Weight,2.097125
DIVISLAB Weight,6.280279
GRASIM Weight,6.074598
HCLTECH Weight,5.686135


In [32]:
pc_sortino=pc.drop(columns=['Sharpe Ratio','Standard Deviation'])

In [33]:
pc_sortino_top10=pc_sortino.sort_values(by=['Sortino Ratio'],ascending=False).head(10)

In [34]:
pc_sortino_top10.to_csv('Sortino_Top10_MV.csv')

In [35]:
pc_sortino_top10

,Returns,Semi-Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
41409,140.692294,2.870183,47.771266,7.814202,2.601109,6.719151,7.765505,0.570514,7.357853,2.311752,...,0.638815,8.056061,0.727291,0.787016,3.549922,7.486257,0.349224,0.399644,0.134699,7.945119
14968,153.381722,3.363632,44.535710,6.193488,2.940444,5.461426,2.609689,0.957942,1.735476,1.426796,...,0.356307,9.031796,1.456404,1.519280,0.554492,7.264488,5.522716,5.566373,1.102003,9.459866
5718,124.703688,2.921402,41.460811,8.984313,0.473520,8.995192,8.011292,3.020958,8.826878,4.060660,...,0.539832,1.551911,3.918613,0.280618,3.770411,8.497334,0.599255,3.593724,0.509921,8.639630
4957,145.667086,3.557677,39.938164,0.022713,2.009030,7.758308,6.634894,5.143949,4.110604,0.071783,...,1.303917,3.757244,0.685046,3.664181,4.235652,9.112047,7.225032,9.366406,3.995841,0.576482
44215,130.822641,3.191000,39.875476,3.541635,0.753263,8.094248,9.275709,1.142133,8.357225,6.891928,...,2.412599,9.977616,1.782759,0.568810,2.335640,0.435457,1.587651,0.983700,3.007030,5.135255
27610,135.022632,3.375584,38.939232,2.680960,1.302068,8.368484,7.611643,2.479078,6.549986,0.353298,...,5.060577,7.260056,1.421244,3.344979,0.798986,7.377264,2.336252,0.154388,4.180205,3.030722
47124,125.156249,3.138692,38.734681,6.362862,7.381729,8.387014,4.189960,3.048284,6.217663,0.311030,...,0.117860,4.294910,3.710959,2.780024,0.067712,6.961660,3.155054,1.388268,2.786116,8.667814
8451,114.632126,2.868899,38.708970,6.489937,4.153954,4.676598,1.103885,2.142047,4.595509,0.820648,...,0.096479,4.728498,0.510703,1.462997,1.523618,4.348963,9.546941,0.205728,2.458957,9.359390
20233,130.394374,3.278665,38.678662,8.685296,6.978577,0.630382,7.742781,2.148297,3.029856,0.481758,...,1.664177,5.448970,1.018731,1.984896,1.126938,8.315074,2.586624,3.554985,3.469859,4.362856
15948,123.836399,3.160411,38.050875,5.683842,5.160299,5.056258,6.581510,0.488983,7.542125,0.089565,...,3.136199,6.943229,1.725725,0.287691,1.993645,7.241348,7.738653,2.109323,7.559798,4.246315


In [36]:
pc_sortino_bottom10=pc_sortino.sort_values(by=['Sortino Ratio'],ascending=False).tail(10)

In [37]:
pc_sortino_bottom10.to_csv('Sortino_Bottom10_MV.csv')

In [38]:
pc_sortino_bottom10

,Returns,Semi-Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
37288,-65.053395,6.845782,-10.025647,7.510234,7.429688,1.253584,2.093017,7.337570,1.440765,6.446985,...,7.894518,0.968515,6.668846,5.908713,3.101023,1.449761,7.973749,7.166598,2.096847,1.986167
28194,-83.243867,8.478438,-10.240549,6.847863,2.672770,2.895737,0.983477,10.977871,1.223388,8.658010,...,6.390513,1.327506,11.119435,0.771624,2.790508,0.060299,2.583817,6.228304,5.282385,3.461821
20294,-74.206148,7.394285,-10.519766,2.713555,3.456304,8.416228,2.677558,1.881608,1.630787,8.106852,...,8.677520,0.750133,7.455356,9.429943,5.195736,1.937820,3.907775,9.356458,3.963274,5.364362
1524,-75.124048,7.383913,-10.658854,0.628032,1.309112,0.408045,4.772428,5.293744,5.863515,8.442065,...,3.976706,0.444499,6.440113,8.844440,6.094837,1.403377,2.891338,8.202434,2.116956,8.140633
1282,-78.299817,7.561657,-10.828290,5.072414,0.357310,0.448620,2.409348,9.075126,1.357226,2.949383,...,8.985492,1.406325,4.301268,8.175435,5.030865,0.076493,8.032165,4.584365,0.382374,8.178052
26278,-71.342147,6.903430,-10.852888,0.176518,7.297542,2.710551,7.396496,3.350008,0.231083,7.133291,...,3.757253,1.185305,6.917186,8.867955,4.959498,0.417242,1.789676,7.518817,1.928631,5.035324
34669,-80.846786,7.285365,-11.588545,8.867263,9.209732,1.346503,2.607462,8.134776,0.369413,4.066250,...,9.963080,2.045640,9.101833,6.517674,9.736605,0.433104,3.004750,1.387363,4.918943,2.856319
37818,-86.620493,7.733542,-11.663542,2.564769,0.170983,0.321117,0.726527,3.638985,3.563850,10.417311,...,4.148226,1.547236,6.395468,7.062064,4.774885,1.887413,10.403251,4.133986,0.703616,9.382860
21401,-80.995074,6.980159,-12.116497,2.901244,4.218665,1.533987,2.449842,5.108351,0.430720,8.936738,...,9.108950,0.202383,6.010384,7.362618,8.725765,1.412608,3.918339,5.146036,4.749056,8.041967
12610,-88.047344,7.453346,-12.293451,8.681919,1.874446,2.966462,4.908225,9.581390,1.171729,10.058346,...,3.540665,1.643762,4.554198,5.638357,6.455982,0.207660,6.120713,4.865574,1.622059,7.523859


In [39]:
sortino_optimal_portfolio=pc_sortino_top10.head(1)

In [40]:
sortino_optimal_portfolio.to_csv('Sortino_Optimal_MV.csv')

In [41]:
sortino_optimal_portfolio.T

,41409
Returns,140.692294
Semi-Deviation,2.870183
Sortino Ratio,47.771266
ASIANPAINT Weight,7.814202
BAJFINANCE Weight,2.601109
BAJAJFINSV Weight,6.719151
BRITANNIA Weight,7.765505
DIVISLAB Weight,0.570514
GRASIM Weight,7.357853
HCLTECH Weight,2.311752


In [42]:
edf=pd.read_csv('n50.csv',parse_dates=['Date'],index_col='Date')
edf = edf.loc["2016-01-01" : ]   
edf.reset_index(drop=True, inplace=True)
(edf / edf.iloc[0]*100).plot(figsize=(10,5))


FileNotFoundError: [Errno 2] No such file or directory: 'n50.csv'

In [ ]:
plt.style.use('seaborn')
pc_sharpe.plot.scatter(x='Standard Deviation', y='Returns', marker='o',figsize=(20, 10), grid=True)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
#GREEN STAR-> Optimal Risky Portfolio
#RED STAR-> Minimum Volatility
plt.subplots(figsize=(20, 10))
plt.xlabel("Expected Volatility")
plt.ylabel("Returns")
plt.scatter(pc_sharpe['Standard Deviation'], pc_sharpe['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(minimum_risk_portfolio['Standard Deviation'], minimum_risk_portfolio['Returns'], color='r', marker='*', s=500)
plt.scatter(sharpe_optimal_portfolio['Standard Deviation'], sharpe_optimal_portfolio['Returns'], color='g', marker='*', s=500)
